In [9]:
# Hier berechne ich einen Distanzmatrix aus den Datensätzen

In [16]:
import pandas as pd
import data_cleanup as dc
import data_exploration as de

In [17]:
fpath = '../DMS_data/P53_HUMAN_Giacomelli_NULL_Etoposide_2018.csv'
df = pd.read_csv(fpath)

In [18]:
# Auftrennen der Mutante in alte und neue AS
mutations_df = dc.aufteilung_mut_pos(df)

In [19]:
# Gruppieren der Substitutionen, geordnet erst nach alter und dann nach neuer AS mit Inspiration von ChatGPT (.groupby in der Pandas Dokumentation beschreibt nur das groupen nach einem Parameter)
subs_df = mutations_df.groupby(["AS_old", "AS_new"])

# Berechnen des means für jede Kombination
mean_scores = subs_df.DMS_score.mean()
# Es gibt 380 Möglichkeiten, 20 AS mit Reihenfolge und ohne zurücklegen zu kombinieren. Aus den Mittelwerten dieser 380 möglichen Substitutionen soll nun eine Distanzmatrix erstellt werden.

In [20]:
# Da "mean_scores" eine Serie ist, muss diese erst wieder in ein Data Frame umgewandelt werden. Dafür müssen den einzelnen Zeilen wieder konkrete Indizes zugewiesen werden, damit wieder ein Data Frame entsteht.
mean_scores_df = mean_scores.reset_index()

# Pivot erstellt aus unserer in einem Data Frame aufgelisteten Substitutionen ein Data Frame, das in Form einer Tabelle eine Übersicht der Mittelwerte zeigt. Die Reihen sind die alten AS, die Spalten die neuen AS.
mean_substitutions = mean_scores_df.pivot(index="AS_old", columns="AS_new", values= "DMS_score")
dc.rmv_na(mean_substitutions)

print(mean_substitutions.columns)

Index(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'],
      dtype='object', name='AS_new')


b### Ergänzungen von Frido für Distanzmatrix

In [21]:
mean_substitutions.head(20)

de.dms_distance_matrix(mean_substitutions)

ValueError: Die Funktion 'aufteilung_mut_pos()' muss vorher ausgeführt werden.

In [22]:
from sklearn.metrics.pairwise import euclidean_distances

# Berechnen der Distanzen der AA zueinander
dms_distances = euclidean_distances(mean_substitutions.values)

# Umwandeln zurück in einen DF zur besseren Übersicht
mean_substitutions_df = pd.DataFrame(dms_distances, index=mean_substitutions.index, columns=mean_substitutions.index)

# Print the first 20 rows of mean_substitutions
mean_substitutions_df.head(20)


In [23]:
mean_substitutions.to_csv('./mean_subst.csv')
mean_substitutions_df.to_csv('./mean_subst_df.csv')

### Kommentar von Enno:
Ich glaube du kannst statt mit mean_distances.T einfach mit der Funktion mean_distances_inverted von Dario arbeiten... Ach und kannst du das ganze noch in ne Funktion umschreiben und in statistical functions importieren? Für das AS Datenset hab ich das schon gemacht. Das brauche ich, damit ich das dann schön und sleak in dem Documentations File einfach als eine Funktion benutzen kann. Das wäre niceeee